# Hyperopt

Github地址 [Hyperopt: Distributed Hyperparameter Optimization](https://github.com/hyperopt/hyperopt).

> hyperopt的官方文档写的并不好，很分散，好在整个package使用起来不是特别复杂。

相关教程文档：
+ [Basic Tutorial](https://github.com/hyperopt/hyperopt/wiki/FMin)
+ [CSDN: Hyperopt 入门指南](https://blog.csdn.net/FontThrone/article/details/85100328)，这份文档写的还不错


hyperopt的使用步骤如下：
1. 定义一个待优化的目标函数，要求是 minimize 类型
2. 定义参数搜索空间
3. 指定存储搜索参数存储的数据库——这一步是可选的
4. 指定搜索算法

## 简单示例

In [1]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.early_stop import no_progress_loss

（1）定义待优化的目标函数

In [17]:
def obj_fun_simple(space):
    """space是参数空间，对应 fmin() 函数中 space= 参数的传入值，可以是dict，也可以是单个值"""
    # print("x_space: ", space)
    x = space
    return (x - 5)**2

（2）定义参数空间

In [18]:
x_space = hp.uniform('x', 0, 10)

（3）执行优化过程，这里跳过了存储数据库

In [19]:
best = fmin(fn=obj_fun_simple, space=x_space, algo=tpe.suggest, max_evals=50)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 819.38trial/s, best loss: 0.0021205822033894383]


In [20]:
print(best.__class__)

<class 'dict'>


In [21]:
best

{'x': 4.953950220376321}

## 复杂示例

定义目标函数时，除了像上面返回一个目标函数值，也可以返回一个dict，用来存放更加丰富的信息。

不过此时需要在返回的dict中设置如下两个特殊的key：
+ `status`，通常是`hyperopt.STATUS_{STRING}`，用于表示此次状态，只有`ok`的记录会被输出
+ `loss`, 浮点数，存放目标函数的值

其他一些可选的key有：
+ `loss_vairance`
+ `true_loss`
+ `true_loss_variance`
+ `attachments`

此外的一个要求就是返回的dict能够被JSON序列化。

为了拿到返回的dict里的自定义信息，需要使用`Trials`对象，它有如下几个封装的属性/方法：
+ `trials`: list of dict, 存放每次迭代时使用的搜索信息
+ `results`: list of dict, 每个dict就是一次迭代中目标函数的返回值
+ `losses()`: list of loss，其中每个loss对应于status=ok的记录
+ `statuses()`: list of status string

In [2]:
def obj_fun(space):
    # print("x_space: ", space)
    x = space
    # 返回一个字典，除了两个必须的key，还有一个要求就是需要能JSON序列化
    return {'loss': (x - 5)**2, 'status': STATUS_OK, 'x_value': x}

In [3]:
x_space = hp.uniform('x', 0, 10)

In [4]:
trials = Trials()

In [6]:
best = fmin(fn=obj_fun, space=x_space, algo=tpe.suggest, max_evals=10, trials=trials)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1999.29trial/s, best loss: 1.52186024686751]


In [7]:
print(best.__class__)

<class 'dict'>


In [8]:
best

{'x': 6.233636999634621}

In [9]:
trials

In [10]:
len(trials.trials)

10

In [12]:
trials.trials[-1]

{'state': 2,
 'tid': 9,
 'spec': None,
 'result': {'loss': 4.4650657730384795,
  'status': 'ok',
  'x_value': 7.113070224350928},
 'misc': {'tid': 9,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [9]},
  'vals': {'x': [7.113070224350928]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2024, 9, 19, 5, 57, 47, 694000),
 'refresh_time': datetime.datetime(2024, 9, 19, 5, 57, 47, 694000)}

In [13]:
len(trials.results)

10

In [14]:
trials.results[-1]

{'loss': 4.4650657730384795, 'status': 'ok', 'x_value': 7.113070224350928}

In [15]:
trials.losses()

[7.756750873823032,
 3.775427069181573,
 5.468446540517656,
 1.52186024686751,
 8.172681854620194,
 2.7049878541541466,
 11.471850048348234,
 2.468664444376138,
 5.9996284069549,
 4.4650657730384795]

In [16]:
trials.statuses()

['ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok']

In [17]:
trials.best_trial

{'state': 2,
 'tid': 3,
 'spec': None,
 'result': {'loss': 1.52186024686751,
  'status': 'ok',
  'x_value': 6.233636999634621},
 'misc': {'tid': 3,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [3]},
  'vals': {'x': [6.233636999634621]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2024, 9, 19, 5, 57, 47, 692000),
 'refresh_time': datetime.datetime(2024, 9, 19, 5, 57, 47, 692000)}

In [18]:
trials.vals

{'x': [2.214905589782811,
  3.0569541772820763,
  7.338470983467115,
  6.233636999634621,
  7.858790278180649,
  6.644684727889861,
  1.6129880353993087,
  3.4288015897487236,
  2.5505861095039695,
  7.113070224350928]}

In [19]:
trials.miscs

[{'tid': 0,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [0]},
  'vals': {'x': [2.214905589782811]}},
 {'tid': 1,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [1]},
  'vals': {'x': [3.0569541772820763]}},
 {'tid': 2,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [2]},
  'vals': {'x': [7.338470983467115]}},
 {'tid': 3,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [3]},
  'vals': {'x': [6.233636999634621]}},
 {'tid': 4,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [4]},
  'vals': {'x': [7.858790278180649]}},
 {'tid': 5,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [5]},
  'vals': {'x': [6.644684727889861]}},
 {'tid': 6,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [6]},
  'vals': {'x': [1.6129880353993087]}

## 参数空间定义

hyperopt里参数空间的定义是通过 `hyperopt.hp` 模块提供的函数实现的，常用函数如下：

+ `hp.randint(label, upper)`    
从 $[0, upper)$ 区间里随机采样。


+ `hp.uniform(label, low, high)`    
从 $[low, high]$ 区间里均匀采样。

+ `hp.quniform(label, low, high, q)`    
返回 round(uniform(low, high) / q) * q 的离散采样值。

+ `hp.loguniform(label, low, high)`    
从 $[exp(low), exp(high)]$ 区间里均匀采样，相当于 exp(uniform(low, high))。

+ `hp.qloguniform(label, low, high, q)`   
相当于 round(exp(uniform(low, high)) / q) * q。

+ `hp.normal(label, mu, sigma)`    
从正态分布 $N(\mu, \sigma)$ 里采样。

+ `hp.qnormal(label, mu, sigma, q)`   
相当于 round(normal(mu, sigma) / q) * q。

+ `hp.lognormal(label, mu, sigma)`   
相当于 exp(normal(mu, sigma))。

+ `hp.qlognormal(label, mu, sigma, q)`   
相当于 round(exp(normal(mu, sigma)) / q) * q 。

+ `hp.choice(label, options)`   
离散采样，`options`是一个list, 随机返回其中的一个项目，对于字符串类型的参数通常使用这个。


上述函数返回只是一个变量，并不是立马完成采样过程，它们只是定义了如何从参数空间中采样，实际采样过程是由 `hyperopt.pyll.stochastic` 模块实现的。

In [31]:
# 实际的参数空间采样过程由下面的模块封装
from hyperopt.pyll.stochastic import sample

In [25]:
space = hp.randint('i', 10)

In [27]:
space

In [28]:
space.name

'hyperopt_param'

In [32]:
sample(space)

array(4, dtype=int64)

+ choice的使用示例

In [39]:
space = hp.choice('p1', ['v1', 'v2', 'v3'])

In [40]:
sample(space)

'v2'

In [42]:
sample(space)

'v1'

+ choice的一个更复杂的例子

In [44]:
space = hp.choice(
    'p2',
    [
        ('case 1', 1 + hp.lognormal('c1', 0, 1)),
        ('case 2', hp.uniform('c2', -10, 10))
    ]
)

In [45]:
sample(space)

('case 2', 7.205905545901672)

In [46]:
sample(space)

('case 1', 1.3548668499043126)

---

# Optuna

Github地址 [Optuna: A hyperparameter optimization framework](https://github.com/optuna/optuna).

相关教程文档：
+ [Optuna英文文档](https://optuna.readthedocs.io/en/stable/)
+ [Optuna中文文档](https://optuna.readthedocs.io/zh-cn/latest/)，这个的版本更新会有些落后，但是也有参考意义

Optuna的使用和Hyperopt不一样，它的参数空间定义是写在目标函数里面的，而不是单独定义。
一般分为3步：
1. 使用 `optuna.study.create_study` 创建一个 `Study` 对象
2. 定义目标函数，并在目标函数里通过`optuna.trial`提供的一系列方法设置参数空间
3. 使用 `optuna.optimize` 运行优化过程，获取结果

Optuna被设计成使用某个数据库作为后端来存储搜索过程，并实现并行搜索的。

In [84]:
from optuna import create_study
from optuna.trial import Trial

（1）定义 Study 对象

In [106]:
# 设置日志输出，可选项
# optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

study_name = "example-study"  # Unique identifier of the study.
# 设置存储的数据库后端
# storage_name = "sqlite:///{}.db".format(study_name)
# 如果设置为 None 或者不传入，就会存放在内存中，不进行持久化
storage_name = None

study = create_study(study_name=study_name, storage=storage_name)

[I 2024-09-19 16:11:36,784] A new study created in memory with name: example-study


In [107]:
study

（2）定义目标函数

In [108]:
def obj_fun_simple(trial: Trial):
    # 目标函数的参数是 trial 对象，里面封装了一系列生产参数空间的方法
    x = trial.suggest_float("x", -10, 10)
    # print(x)
    # 使用 Trial 对象记录自定义的信息
    trial.set_user_attr('x_value', x)
    return (x - 2) ** 2

（3）运行优化过程

In [110]:
study.optimize(obj_fun_simple, n_trials=5)

[I 2024-09-19 16:11:44,328] Trial 0 finished with value: 4.053126933147154 and parameters: {'x': -0.013237922637847177}. Best is trial 0 with value: 4.053126933147154.
[I 2024-09-19 16:11:44,329] Trial 1 finished with value: 42.482525812898935 and parameters: {'x': 8.517862058443622}. Best is trial 0 with value: 4.053126933147154.
[I 2024-09-19 16:11:44,330] Trial 2 finished with value: 27.210341633410053 and parameters: {'x': 7.21635328878423}. Best is trial 0 with value: 4.053126933147154.
[I 2024-09-19 16:11:44,331] Trial 3 finished with value: 0.3386616872704957 and parameters: {'x': 2.5819464642649663}. Best is trial 3 with value: 0.3386616872704957.
[I 2024-09-19 16:11:44,332] Trial 4 finished with value: 29.65044738811833 and parameters: {'x': 7.445222436973381}. Best is trial 3 with value: 0.3386616872704957.


In [111]:
len(study.trials)

5

In [112]:
study.best_value

0.3386616872704957

In [113]:
study.best_params

{'x': 2.5819464642649663}

In [114]:
study.best_trial

FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.3386616872704957], datetime_start=datetime.datetime(2024, 9, 19, 16, 11, 44, 331595), datetime_complete=datetime.datetime(2024, 9, 19, 16, 11, 44, 331595), params={'x': 2.5819464642649663}, user_attrs={'x_value': 2.5819464642649663}, system_attrs={}, intermediate_values={}, distributions={'x': FloatDistribution(high=10.0, log=False, low=-10.0, step=None)}, trial_id=3, value=None)

In [117]:
study.best_trial.user_attrs

{'x_value': 2.5819464642649663}

In [115]:
study.best_trials

[FrozenTrial(number=3, state=TrialState.COMPLETE, values=[0.3386616872704957], datetime_start=datetime.datetime(2024, 9, 19, 16, 11, 44, 331595), datetime_complete=datetime.datetime(2024, 9, 19, 16, 11, 44, 331595), params={'x': 2.5819464642649663}, user_attrs={'x_value': 2.5819464642649663}, system_attrs={}, intermediate_values={}, distributions={'x': FloatDistribution(high=10.0, log=False, low=-10.0, step=None)}, trial_id=3, value=None)]

In [116]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_x,user_attrs_x_value,state
0,0,4.053127,2024-09-19 16:11:44.326595,2024-09-19 16:11:44.327594,0 days 00:00:00.000999,-0.013238,-0.013238,COMPLETE
1,1,42.482526,2024-09-19 16:11:44.329595,2024-09-19 16:11:44.329595,0 days 00:00:00,8.517862,8.517862,COMPLETE
2,2,27.210342,2024-09-19 16:11:44.329595,2024-09-19 16:11:44.329595,0 days 00:00:00,7.216353,7.216353,COMPLETE
3,3,0.338662,2024-09-19 16:11:44.331595,2024-09-19 16:11:44.331595,0 days 00:00:00,2.581946,2.581946,COMPLETE
4,4,29.650447,2024-09-19 16:11:44.332595,2024-09-19 16:11:44.332595,0 days 00:00:00,7.445222,7.445222,COMPLETE
